# Statistics
This notebooks allows you to obtain some statistics of the dataset.

In [ ]:
!pip install pyforest==0.1.1
#https://pypi.org/project/pyforest/0.1.1/
import pyforest  #With this library, you won't need to import more packages!!!


     |████████████████████████████████| 3.4MB 6.5MB/s 
  Created wheel for pyforest: filename=pyforest-0.1.1-py2.py3-none-any.whl size=9213 sha256=e93e1b16570bceb4ee5e560d59e406fd016e03cef9e32bcd258082541327cd2d
  Stored in directory: /root/.cache/pip/wheels/77/f9/78/51500678d6ce472b574216a40cba6c81d1766ee7cc838cce3c
Successfully built pyforest


In [1]:
#from google.colab import drive
#drive.mount("/content/drive/")

#root='drive/My Drive/Colab Notebooks/nlp4rareNER/'
#path=root+'data/goldstandard/'

path = '../corpus/brat/'

train_path=path+'train/'
test_path=path+'test/'

print("training:", train_path)
print("test:", test_path)

training: ../corpus/brat/train/
test: ../corpus/brat/test/


## Number of documents, sentences and tokens

In [2]:
import glob
size_train=len(glob.glob(train_path+'*.txt'))
print('number of documents in training:',size_train)
size_test=len(glob.glob(test_path+'*.txt'))
print('number of documents in test:',size_test)
total=size_train+size_test
print('total of documents:',total)


number of documents in training: 100
number of documents in test: 26
total of documents: 126


We will install Spacy, a library NLP, that helps us to tokenize the texts.

In [9]:
!python -m spacy download en_core_web_sm #install spacy and download the model en_core_web_sm
import spacy #NLP library for sentence segmentation and tokenization
#nlp = spacy.load("pt_core_news_lg")
!python -m spacy download pt_core_news_sm
nlp = spacy.load("pt_core_news_sm")

2022-08-13 09:03:20.084669: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2022-08-13 09:03:20.084702: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-13 09:03:21.976358: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-08-13 09:03:21.976392: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-13 09:03:21.980304: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: TerumiNB
2022-08-13 09:03:21.980448: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: TerumiNB
ERROR: Invalid requirement: '#install'
You should consider upgrading via the 'C:\Users\lisat\anaconda3\python.exe -m pip

  Attempting uninstall: pt-core-news-sm
    Found existing installation: pt-core-news-sm 3.0.0
    Uninstalling pt-core-news-sm-3.0.0:
      Successfully uninstalled pt-core-news-sm-3.0.0
[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


2022-08-13 09:03:27.371995: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2022-08-13 09:03:27.372028: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-13 09:03:29.172040: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-08-13 09:03:29.172071: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-13 09:03:29.175607: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: TerumiNB
2022-08-13 09:03:29.175728: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: TerumiNB
You should consider upgrading via the 'C:\Users\lisat\anaconda3\python.exe -m pip install --upgrade pip' command.


In [13]:
def getText(path_file):
    """returns the text of the file path_file"""
    f=open(path_file,'r', encoding='utf-8')
    text=f.read()
    f.close()
    return text

def countTokens(path):
    """This functions takes a directory path where a dataset is stored, g
    gets its txt files to parse them  by using Spacy 
    for obtaining the number of tokens and sentences in
    the dataset"""
    
    total_sentences=0
    total_tokens=0
    files=glob.glob(path+'*.txt')
    #print(path,len(files))
    
    for i,file_text in enumerate(files):
        text=getText(file_text)
        #print(file_text,text)

        doc = nlp(text)
        total_sentences+=len(list(doc.sents))
        total_tokens+=len(doc)

    return total_sentences, total_tokens

sentences_train, tokens_train = countTokens(train_path)
sentences_test, tokens_test = countTokens(test_path)

print('Sentences and tokens in training: ', sentences_train, tokens_train)
print('Sentences and tokens in text: ', sentences_test, tokens_test)


Sentences and tokens in training:  2273 17154
Sentences and tokens in text:  693 5310


## Number of entities

We now obtain the number of entities:

In [24]:
def getAnnotations(path_file,allTypes=False):
    """returns a dataframe with its annotations"""
    annotations=[]
    try:  
        f=open(path_file,'r')
        annotations = f.readlines()       
        f.close()
    except:
        pass

    num_entities=0
    total_problema=0
    total_teste=0
    total_tratamento=0
    total_anatomia=0

    for ann in annotations:
        if ann.startswith('T'):
            idEnt=ann[:ann.index('\t')]
            typeEnt=ann[ann.index('\t')+1:ann.index(' ')]
            num_entities+=1

            if typeEnt=='Problema':
                if not allTypes:
                    total_problema+=1
                else:
                    num_entities-=1
            elif typeEnt=='Teste':
                total_teste+=1
            elif typeEnt=='Tratamento':
                total_tratamento+=1
            elif typeEnt=='Anatomia':
                total_anatomia+=1
            else:
                #if allTypes==False
                print('ERRO, typeEnt:', typeEnt)
                pass

    return num_entities, total_problema, total_teste, total_tratamento, total_anatomia

def getAnnotationsDescontinuas(path_file,allTypes=False):
    """returns a dataframe with its annotations"""
    annotations=[]
    try:  
        f=open(path_file,'r')
        annotations = f.readlines()       
        f.close()
    except:
        pass

    num_entities=0
    total_problema=0
    total_teste=0
    total_tratamento=0
    total_anatomia=0

    for ann in annotations:
        if ann.startswith('T'):
            idEnt=ann[:ann.index('\t')]
            typeEnt=ann[ann.index('\t')+1:ann.index(' ')]
            listaAnn = ann.split('\t')
            indices=listaAnn[1]
            if ';' in indices:
                # ex T10	Problema 244 252;283 306	dispneia mdoeardos-leves esforço
                num_entities+=1
                if typeEnt=='Problema':
                    total_problema+=1
                elif typeEnt=='Teste':
                    total_teste+=1
                elif typeEnt=='Tratamento':
                    total_tratamento+=1
                elif typeEnt=='Anatomia':
                    total_anatomia+=1
                else:
                    #if allTypes==False
                    print('ERRO, typeEnt:', typeEnt)
                    pass

    return num_entities, total_problema, total_teste, total_tratamento, total_anatomia


def countEntities(path):
    total_entities=0
    total_problema=0
    total_teste=0
    total_tratamento=0
    total_anatomia=0
    
    files=glob.glob(path+'*.ann')
    #print(path,len(files))
    
    for i,file_path in enumerate(files):
        ne, td, tr, tsk, tsy=getAnnotations(file_path)
        total_entities+=ne
        total_problema+=td
        total_teste+=tr
        total_tratamento+=tsk
        total_anatomia+=tsy

    return total_entities, total_problema, total_teste, total_tratamento, total_anatomia


def countEntitiesDescontinuas(path):
    total_entities=0
    total_problema=0
    total_teste=0
    total_tratamento=0
    total_anatomia=0
    
    files=glob.glob(path+'*.ann')
    #print(path,len(files))
    
    for i,file_path in enumerate(files):
        ne, td, tr, tsk, tsy=getAnnotationsDescontinuas(file_path)
        total_entities+=ne
        total_problema+=td
        total_teste+=tr
        total_tratamento+=tsk
        total_anatomia+=tsy

    return total_entities, total_problema, total_teste, total_tratamento, total_anatomia

print('Entities,Problema,Teste, Tratamento, Anatomia')
total_entities, total_problema, total_teste, total_tratamento, total_anatomia= countEntities(train_path)
print(total_entities, total_problema, total_teste, total_tratamento, total_anatomia)

total_entities, total_problema, total_teste, total_tratamento, total_anatomia = countEntities(test_path)
print(total_entities, total_problema, total_teste, total_tratamento, total_anatomia)

print('\n--Descontinuas--\n')
print('Entities,Problema,Teste, Tratamento, Anatomia')
total_entities, total_problema, total_teste, total_tratamento, total_anatomia= countEntitiesDescontinuas(train_path)
print(total_entities, total_problema, total_teste, total_tratamento, total_anatomia)

total_entities, total_problema, total_teste, total_tratamento, total_anatomia = countEntitiesDescontinuas(test_path)
print(total_entities, total_problema, total_teste, total_tratamento, total_anatomia)


Entities,Problema,Teste, Tratamento, Anatomia
2188 780 523 528 357
651 222 150 140 139

--Descontinuas--

Entities,Problema,Teste, Tratamento, Anatomia
2188 780 523 528 357
651 222 150 140 139
